In [2]:
pip install QuantLib

Note: you may need to restart the kernel to use updated packages.


In [16]:
import os
import math
import numpy as np
import pandas as pd
import QuantLib as ql
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')
import multiprocessing
import logging
logging.getLogger().setLevel(logging.DEBUG)
np.random.seed(42)
nasdaq_holidays = [
    '2018-01-01', '2018-01-15', '2018-02-19', '2018-03-30', '2018-05-28',
    '2018-07-04', '2018-09-03', '2018-11-22', '2018-12-25',
    '2019-01-01', '2019-01-21', '2019-02-18', '2019-04-19', '2019-05-27',
    '2019-07-04', '2019-09-02', '2019-11-28', '2019-12-25',
    '2020-01-01', '2020-01-20', '2020-02-17', '2020-04-10', '2020-05-25',
    '2020-07-03', '2020-09-07', '2020-11-26', '2020-12-25',
    '2021-01-01', '2021-01-18', '2021-02-15', '2021-04-02', '2021-05-31',
    '2021-07-05', '2021-09-06', '2021-11-25', '2021-12-24',
    '2022-01-01', '2022-01-17', '2022-02-21', '2022-04-15', '2022-05-30',
    '2022-07-04', '2022-09-05', '2022-11-24', '2022-12-26',
    '2023-01-01', '2023-01-16', '2023-02-20', '2023-04-07', '2023-05-29',
    '2023-07-04', '2023-09-04', '2023-11-23', '2023-12-25'
]
nasdaq_holidays = np.array(nasdaq_holidays, dtype='datetime64[D]')


def IR_effect(init_date, start_date, end_date):
    if start_date == end_date:
        return 0.0
    else:
        tau = np.busday_count(start_date, end_date, holidays=nasdaq_holidays) / 252
        if tau == 0.0:
            return 0.0
        else:
            beta0 = IRParams.loc[IRParams.Date <= init_date, 'BETA0'].values[-1]
            beta1 = IRParams.loc[IRParams.Date <= init_date, 'BETA1'].values[-1]
            beta2 = IRParams.loc[IRParams.Date <= init_date, 'BETA2'].values[-1]
            tau1 = IRParams.loc[IRParams.Date <= init_date, 'TAU1'].values[-1]

            r = beta0 + beta1*(1-math.exp(-tau/tau1))/(tau/tau1) + beta2*((1-math.exp(-tau/tau1))/(tau/tau1)-math.exp(-tau/tau1))
            return r/100


def date_asQuantLib(t):
    result = pd.to_datetime(t)
    return ql.Date(result.day, result.month, result.year)


def read_all_paths(directory):
    all_paths = []
    for root, dirs, files in os.walk(directory):
        for name in files:
            file_path = os.path.join(root, name)
            all_paths.append(file_path)
        for name in dirs:
            dir_path = os.path.join(root, name)
            all_paths.append(dir_path)
    return all_paths


# Define a helper to price options using the Heston model
def model_price(strike, expiry, model):
    payoff = ql.PlainVanillaPayoff(ql.Option.Call, strike)
    exercise = ql.EuropeanExercise(expiry)
    option = ql.VanillaOption(payoff, exercise)
    engine = ql.AnalyticHestonEngine(model)
    if engine is None:
        print("Pricing engine is not initialized properly.")
    option.setPricingEngine(engine)
    return option.NPV()


# Define the objective function (least squares + Tikhonov regularization)
def objective_function(params, market_data, alpha, prior_params):
    v0, kappa, theta, sigma, rho = params

    strikes = market_data['strike_prices']
    market_prices = market_data['market_prices']
    expiries = [date_asQuantLib(t) for t in market_data['expiry']]
    rf = market_data['risk_free_curve']

    heston_process = ql.HestonProcess(ql.YieldTermStructureHandle(rf),
                                      market_data['dividend_yield'], market_data['spot_price'],
                                      v0, kappa, theta, sigma, rho)
    heston_model = ql.HestonModel(heston_process)

    # Calculate the sum of squared errors between model and market prices
    errors = np.sum([(model_price(strike, expiry, heston_model) - market_price) ** 2
                     for strike, market_price, expiry in zip(strikes, market_prices, expiries)])

    # Tikhonov regularization (penalty for deviating from prior guess)
    regularization = alpha * np.sum((params - prior_params) ** 2)

    return errors + regularization


# Calibrate the parameters on each t0
def calibrateHestonT0(df, t0, S0, r0, init_params):
    valuation_date = t0
    ql.Settings.instance().evaluationDate = valuation_date
    spot_handle = ql.QuoteHandle(ql.SimpleQuote(S0))
    dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(valuation_date, 0.0, day_count))
    risk_free_curve = ql.FlatForward(valuation_date, ql.QuoteHandle(ql.SimpleQuote(r0)), day_count)

    market_data = {
        'strike_prices': df['K'],
        'market_prices': (df['ask'] + df['bid']) / 2,  # Observed market option prices
        'expiry': df['t'],
        'spot_price': spot_handle,
        'dividend_yield': dividend_yield,
        'risk_free_curve': risk_free_curve
    }

    initial_guess = init_params
    prior_params = init_params  # Prior guess for Tikhonov
    alpha = 0.01  # Regularization parameter

    result = minimize(
        objective_function,
        x0=initial_guess,
        args=(market_data, alpha, prior_params),
        method='L-BFGS-B',  # Optimization method, can use other methods
        bounds=[(0.01, 0.2), (0.1, 10), (0.001, 0.2), (0.001, 10.0), (-0.9, 0.9)]  # Bounds on parameters
    )

    return result.x

In [24]:
if __name__ == "__main__":
    v0 = 0.04  # Initial variance
    kappa = 0.8  # Rate of mean reversion
    theta = 0.04  # Long-term variance
    sigma = 0.1  # Volatility of volatility
    rho = -0.2  # Correlation (can be negative)
    init_params = np.array([v0, kappa, theta, sigma, rho])

    IRParams = pd.read_csv('interest_rates_parameters.csv', parse_dates=['Date'], dayfirst=True)
    IRParams = IRParams.fillna(method='ffill').sort_values(by='Date')
    IRParams.head()

    calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
    day_count = ql.Actual365Fixed()

    tickers = ['JNJ','AMZN','GOOGL','JPM','MSFT','PG','TSLA','V','WMT']

    params_all = pd.DataFrame()
    for ticker in tickers:
        stock_df = pd.read_csv('data/adjusted_Stock_Daily/{}_stock_daily_adjusted.csv'.format(ticker.lower()), index_col=0)
        stock_df = stock_df.set_index('date')
    
        data = pd.read_csv(f'rawImpl_localVol_{ticker.upper()}.csv',index_col=0)
        data.columns = ['t0', 't', 'K', 'ask', 'bid','impl_volatility']
        data['mid_price'] = data[['ask','bid']].mean(axis=1)

        t0List = data['t0'].unique()
        for t0 in t0List:
            df = data[data['t0'] == t0]
            df = df[df['t'] > t0]
            S0 = stock_df.loc[t0]['adjusted_price']
            r = IR_effect(t0, t0, t0)
            params = calibrateHestonT0(df, date_asQuantLib(t0), S0, r, init_params)
            temp = pd.DataFrame(params, columns=pd.MultiIndex.from_tuples([(ticker, t0)])).T
            params_all = pd.concat([params_all, temp])

KeyboardInterrupt: 

In [ ]:
params_all.columns = ['v0', 'kappa', 'theta', 'sigma', 'rho']
params_all.to_csv('hestonParams.csv')

In [27]:
params_all

0         1         2         3         4
JNJ 2018-01-02  0.2  0.100000  0.200000  0.504163 -0.900000
    2018-01-03  0.2  0.337423  0.001000  0.403650 -0.900000
    2018-01-04  0.2  0.837312  0.006793  0.086482 -0.209276
    2018-01-05  0.2  0.100000  0.200000  0.472792 -0.900000
    2018-01-08  0.2  1.264119  0.001000  0.001000  0.000255
...             ...       ...       ...       ...       ...
    2018-12-31  0.2  0.100000  0.200000  0.432834 -0.900000
    2019-01-02  0.2  0.678798  0.001000  0.179264 -0.900000
    2019-01-03  0.2  0.176936  0.001000  0.306813 -0.900000
    2019-01-04  0.2  0.100000  0.200000  0.427287 -0.900000
    2019-01-07  0.2  0.100000  0.001000  0.373955 -0.900000

[255 rows x 5 columns]